# **Dog vs Cat Predection Project**

> Import All The Lib... 

    1. Numpy 
    2. Tensorflow
    3. Keras 

> Dataset --> https://www.kaggle.com/c/dogs-vs-cats/data

In [83]:
'''
    Import all the Lib required....
'''

import tensorflow as tf
import numpy as np
import keras

# Check the model 

print(f"Keras  : {keras.__version__}")
print(f"Tensorflow  : {tf.__version__}")
print(f"Numpy  : {np.__version__}")

Keras  : 3.5.0
Tensorflow  : 2.17.0
Numpy  : 1.26.4


In [84]:
'''
    Generate and rescale the image using ImageDataGenerator from Tensorflow.....
'''

# Path

test_path = r"E:\ml\Project\Cat vs Dog\Data\test_set"
train_path = r"E:\ml\Project\Cat vs Dog\Data\training_set"
input_size = keras.layers.Input(shape=(224,224,3))


training_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    height_shift_range = 0.2,
    width_shift_range = 0.2,
    horizontal_flip = True
)

Validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Dataset...

training_dataset = training_datagen.flow_from_directory(
    train_path,
    batch_size=16,
    target_size = (224,224),
    class_mode = 'categorical'
)

validation_dataset = Validation_datagen.flow_from_directory(
    test_path,
    batch_size = 16,
    target_size = (224,224),
    class_mode = 'categorical'
)

# 40% of work is done.....

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
# Model Creation....

input_shape = keras.layers.Input(shape=(224,224,3))

model = keras.applications.VGG16(input_tensor=input_shape,include_top=True,weights='imagenet')
# model.summary()
lastlayer = model.get_layer('block5_pool').output
x = keras.layers.Flatten(name='flatten')(lastlayer)
x = keras.layers.Dense(128,activation='relu',name='d1')(x)
x = keras.layers.Dense(128,activation='relu',name='d2')(x)

# bcz we need 2 output 
# cat & Dog
x = keras.layers.Dense(2,activation='softmax',name='Output')(x)

model = keras.Model(input_shape,x)
# model.summary()


for layer in model.layers[:-4]:
    layer.trainable = False
for i  in model.layers:
    # print(f"{i.name}  :  {i.trainable}")
    pass


# train on last 4 layers....

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

es = keras.callbacks.EarlyStopping(monitor='val_loss',patience=3)

model.fit(
    training_dataset,
    validation_data=validation_dataset,
    steps_per_epoch=8000//16,
    validation_steps=2000//16,
    callbacks=es,
    epochs=5
)



In [106]:
def cd(t_path):
    sample_image = keras.preprocessing.image.load_img(t_path,target_size=(224,224))
    sample_image = keras.preprocessing.image.img_to_array(sample_image)
    sample_image = np.expand_dims(sample_image,axis=0)
    pred = model.predict(sample_image)

    return pred



### **Prediction Time...**

In [120]:
training_dataset.class_indices
sample_path = r"E:\ml\Project\Cat vs Dog\Data\test_set\dogs\dog.4989.jpg"

cd(sample_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


'Dog'